In [196]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_samples
from sklearn.metrics import log_loss
from sklearn.model_selection._split import KFold
from sklearn.ensemble import RandomForestClassifier
#- - - - - - - -- - - - - - - - - - - - -- - - - - - - - - - - - - -- - - - - - - - - - - - -- - -
import warnings
warnings.filterwarnings("ignore")

In [175]:
# Import data from assignment 4 and set y values
full_df = pd.read_csv('full_dataset.csv')
y=full_df['excess returns']
y[y<0]=-1
y[y>=0]=1
full_df = full_df[full_df['yyyymm']>194412].copy()
full_df = full_df.iloc[:,1:].copy()
full_df = full_df.drop(columns=['yyyymm', 'Index','CRSP_SPvw','CRSP_SPvwx', 
                                'D12','E12','AAA', 'corpr','BAA', 'Rfree', 'csp', 'excess returns'])
full_df.reset_index(drop=True, inplace=True)
corr_df = full_df.iloc[:,:-1].corr()

In [409]:
def clusterKMeansBase(corr0, seed, maxNumClusters=10,n_init=10):
    
    x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
    for init in range(n_init):
        for i in range(2,maxNumClusters+1):
            kmeans_=KMeans(n_clusters=i,n_jobs=1,n_init=1, random_state=seed)
            kmeans_=kmeans_.fit(x)
            silh_=silhouette_samples(x,kmeans_.labels_)
            stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
            if np.isnan(stat[1]) or stat[0]>stat[1]:
                silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
    corr1=corr0.iloc[newIdx] # reorder rows
    corr1=corr1.iloc[:,newIdx] # reorder columns
    clstrs={i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() \
            for i in np.unique(kmeans.labels_) } # cluster members
    silh=pd.Series(silh,index=x.index)
    return corr1,clstrs,silh

In [410]:
def featImpMDA_Clustered(clf,X,y,clstrs,n_splits=10):

    cvGen=KFold(n_splits=n_splits)
    scr0,scr1=pd.Series(),pd.DataFrame(columns=clstrs.keys())
    for i,(train,test) in enumerate(cvGen.split(X=X)):
        X0,y0=X.iloc[train,:],y.iloc[train]
        X1,y1=X.iloc[test,:],y.iloc[test]
        fit=clf.fit(X=X0,y=y0)
        prob=fit.predict_proba(X1)
        scr0.loc[i]=-log_loss(y1,prob,labels=clf.classes_)
        for j in scr1.columns:
            X1_=X1.copy(deep=True)
            for k in clstrs[j]:
                np.random.shuffle(X1_[k].values) # shuffle cluster
            prob=fit.predict_proba(X1_)
            scr1.loc[i,j]=-log_loss(y1,prob,labels=clf.classes_)
    
    imp=(-1*scr1).add(scr0,axis=0)
    
    imp=imp/(-1*scr1)
    imp=pd.concat(
                  {'mean':imp.mean(),'std':imp.std()*imp.shape[0]**-.5},
                  axis=1
    )
    imp.index=['C_'+str(i) for i in imp.index]
    return imp

# Question 1
Set the random seed to 1. Apply the distance metric based on correlations to the features used in Modules 5-7 in predicting the sign of SPX monthly returns, and apply K-means clustering to those features, using an optimal number of clusters as defined in MLAM section 4.4.2. Use Snippet 4.1 in MLAM. How many clusters did you find? What are the top 2 clusters? (2 pt)

## Answer
Three clusters, top importance cluster is cluster 1 followed by cluster 0. I tried to create a normalized importance score like Dr Chan used in his paper with Man, but wasn't sure how they accomplished it with frequent negative values. A trick that I've used is adding a constant (usually 1) but that didn't work due to the small magnitude of these numbers, so I added the max value to each value in the means vector to allow a sum to unity. 

In [415]:
corr1,clstrs,silh = clusterKMeansBase(corr_df, seed=1)
print(f'Clusters:')
display(clstrs)
print(f'Silhouette\n{silh}')

Clusters:


{0: ['ltr', 'svar', 'dfy', 'dfr'],
 1: ['b/m', 'd/p', 'd/y', 'e10/p'],
 2: ['tbl', 'lty', 'ntis', 'infl', 'e/p']}

Silhouette
b/m      0.386785
tbl      0.315759
lty      0.166338
ntis     0.114055
infl    -0.066062
ltr      0.313463
svar     0.315946
dfy      0.276391
dfr      0.359036
d/p      0.634007
d/y      0.543941
e/p      0.033510
e10/p    0.616197
dtype: float64


In [420]:
clf = RandomForestClassifier(n_estimators=100, max_depth=4 ,random_state=1)
imp = featImpMDA_Clustered(clf,X_test,y_test,clstrs,n_splits=10)

In [421]:
# Adding max value to prevent negatives from affecting score
# Tried adding 1 but causes near equal importance scores bc of magnitude
# Wasn't able to find an explicit example for Importance Scores
mean = imp['mean'] +imp['mean'].max() 
imp['Importance Scores']=mean/mean.sum()
imp

,mean,std,Importance Scores
C_0,0.000348,0.003602,0.252973
C_1,0.016275,0.012482,0.495363
C_2,0.000262,0.007512,0.251664


# Question 2

Set the random seed to 2. Repeat Part 1. Did the number of clusters change from what you found in Part 1? Did the top 2 clusters change? (I.e. Do they have identical constituents?) (2 pt)

## Answer
Number of clusters did not change nor did the members in each cluster, but the top two cluster's feature importance scores did change. 

Clusters are the same with seed two. The top cluster is cluster 1 again however cluster 2 has the second highest importance score. 

In [422]:
corr1,clstrs,silh = clusterKMeansBase(corr_df, seed=2)
print(f'Clusters:')
display(clstrs)
print(f'Silhouette\n{silh}')

Clusters:


{0: ['ltr', 'svar', 'dfy', 'dfr'],
 1: ['b/m', 'infl', 'd/p', 'd/y', 'e10/p'],
 2: ['tbl', 'lty', 'ntis', 'e/p']}

Silhouette
b/m      0.350050
tbl      0.367971
lty      0.219168
ntis     0.066823
infl     0.066062
ltr      0.312200
svar     0.327805
dfy      0.278676
dfr      0.334803
d/p      0.542574
d/y      0.471427
e/p      0.009710
e10/p    0.530646
dtype: float64


In [423]:
clf = RandomForestClassifier(n_estimators=100, max_depth=4 ,random_state=2)
imp = featImpMDA_Clustered(clf,X_test,y_test,clstrs,n_splits=10)

In [424]:
# Adding max value to prevent negatives from affecting score
# Tried adding 1 but causes near equal importance scores bc of magnitude
mean = imp['mean'] +imp['mean'].max() 
imp['Importance Scores']=mean/mean.sum()
imp

,mean,std,Importance Scores
C_0,0.003968,0.008052,0.252573
C_1,0.011085,0.012012,0.341914
C_2,0.016152,0.010691,0.405512


# Question 3

Apply hierarchical clustering to the same features, as per Chan and Man 2021b (see article on Overview Page). Again, set the random seed to 1. How many clusters did you find?  What are the top 2 clusters? (3 pt)

## Answer:
Two clusters that are very close to the Man and Chan 2021 paper. Top cluster is cluster zero and cluster 1 follows. 

In [442]:
def clusterAgglomBase(corr0, seed, maxNumClusters=10,n_init=10):
    np.random.seed(seed)
    x,silh=((1-corr0.fillna(0))/2.)**.5,pd.Series()# observations matrix
    for init in range(n_init):
        for i in range(2,maxNumClusters+1):
            kmeans_=AgglomerativeClustering(n_clusters=i)
            kmeans_=kmeans_.fit(x)
            silh_=silhouette_samples(x,kmeans_.labels_)
            stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
            if np.isnan(stat[1]) or stat[0]>stat[1]:
                silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
    corr1=corr0.iloc[newIdx] # reorder rows
    corr1=corr1.iloc[:,newIdx] # reorder columns
    clstrs={i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() \
            for i in np.unique(kmeans.labels_) } # cluster members
    silh=pd.Series(silh,index=x.index)
    return corr1,clstrs,silh

In [443]:
corr1,clstrs,silh = clusterAgglomBase(corr_df, seed=1, n_init=10)
print(f'Clusters:')
display(clstrs)
print(f'Silhouette\n{silh}')

Clusters:


{0: ['b/m', 'tbl', 'lty', 'ntis', 'infl', 'd/p', 'd/y', 'e/p', 'e10/p'],
 1: ['ltr', 'svar', 'dfy', 'dfr']}

Silhouette
b/m      0.367031
tbl      0.142604
lty     -0.014856
ntis     0.096808
infl     0.143748
ltr      0.372457
svar     0.358123
dfy      0.317302
dfr      0.359797
d/p      0.325710
d/y      0.373252
e/p      0.334798
e10/p    0.299608
dtype: float64


In [444]:
clf = RandomForestClassifier(n_estimators=100, max_depth=4 ,random_state=1)
imp = featImpMDA_Clustered(clf,X_test,y_test,clstrs,n_splits=10)

In [445]:
# Adding max value to prevent negatives from affecting score
# Tried adding 1 but causes near equal importance scores bc of magnitude
mean = imp['mean'] +imp['mean'].max() 
imp['Importance Scores']=mean/mean.sum()
imp

,mean,std,Importance Scores
C_0,0.018506,0.018368,0.809911
C_1,-0.009819,0.005459,0.190089


# Question 4
Set the random seed to 2. Repeat Part 3. How many clusters did you find?  What are the top 2 clusters? (3 pt)

## Answer:
Two clusters. Top cluster is cluster zero and cluster 1 follows. Importance scores are very close to answers from question 3. 

In [439]:
corr1,clstrs,silh = clusterAgglomBase(corr_df, seed=2, n_init=10)
print(f'Clusters:')
display(clstrs)
print(f'Silhouette\n{silh}')

Clusters:


{0: ['b/m', 'tbl', 'lty', 'ntis', 'infl', 'd/p', 'd/y', 'e/p', 'e10/p'],
 1: ['ltr', 'svar', 'dfy', 'dfr']}

Silhouette
b/m      0.367031
tbl      0.142604
lty     -0.014856
ntis     0.096808
infl     0.143748
ltr      0.372457
svar     0.358123
dfy      0.317302
dfr      0.359797
d/p      0.325710
d/y      0.373252
e/p      0.334798
e10/p    0.299608
dtype: float64


In [440]:
clf = RandomForestClassifier(n_estimators=100, max_depth=4 ,random_state=2)
imp = featImpMDA_Clustered(clf,X_test,y_test,clstrs,n_splits=10)

In [441]:
# Adding max value to prevent negatives from affecting score
# Tried adding 1 but causes near equal importance scores bc of magnitude
mean = imp['mean'] +imp['mean'].max() 
imp['Importance Scores']=mean/mean.sum()
imp

,mean,std,Importance Scores
C_0,0.010356,0.015008,0.822245
C_1,-0.005878,0.005660,0.177755
